<a href="https://colab.research.google.com/github/BuxGuerra/TPICD/blob/main/analiseFinalista.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Prevendo finalistas da NBA

O nosso objetivo neste notebook é prever se um time da NBA irá chegar à final dos playofs analisando suas estatísticas durante a temporada regular.

Começaremos com a preparação e o pré-processamento dos dados.

In [74]:
import pandas as pd
import numpy as np
import os
from sklearn.metrics import (confusion_matrix, accuracy_score)
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
import seaborn
import matplotlib.pyplot as plt
import statsmodels.api as sm

##Preparação dos dados

###Concatenar bases e rotular

Nessa etapa temos dois objetivos:



*   Identificar os finalistas
*   Unificar as bases de dados

Como dito no notebook importanciaCaracteristicas.ipynb, cada base de dados que temos representa uma temporada regular da NBA. As linhas das tabelas representam os times e as colunas nos fornecem estatísticas sumarizadas dos times ao longo da temporada regular.

In [75]:
!git clone https://github.com/BuxGuerra/TPICD.git

dfTemporada2010 = pd.read_csv("/content/TPICD/data/basesSeparadas/2010-11.csv")
dfTemporada2010.head()

fatal: destination path 'TPICD' already exists and is not an empty directory.


,Unnamed: 0,TEAM_ID,TEAM_NAME,GP,W,L,W_PCT,MIN,FGM,FGA,...,REB_RANK,AST_RANK,TOV_RANK,STL_RANK,BLK_RANK,BLKA_RANK,PF_RANK,PFD_RANK,PTS_RANK,PLUS_MINUS_RANK
0,0,1610612737,Atlanta Hawks,82,44,38,0.537,48.1,36.2,78.4,...,28,12,9,29,28,5,1,29,26,16
1,1,1610612738,Boston Celtics,82,56,26,0.683,48.2,36.9,75.8,...,29,4,22,3,27,10,15,22,23,6
2,2,1610612766,Charlotte Bobcats,82,34,48,0.415,48.3,35.0,77.6,...,24,18,21,28,8,29,8,13,29,24
3,3,1610612741,Chicago Bulls,82,62,20,0.756,48.4,37.1,80.3,...,2,9,17,17,5,26,9,24,20,2
4,4,1610612739,Cleveland Cavaliers,82,19,63,0.232,48.2,35.2,81.1,...,22,20,18,26,28,28,12,10,25,30


In [76]:
#Dicionário para guardar finalistas:
seasonFirstSecond = {'2006-2007':('San Antonio Spurs','Cleveland Cavaliers'),
                     '2007-2008':('Boston Celtics','Los Angeles Lakers'),
                     '2008-2009':('Los Angeles Lakers','Orlando Magic'),
                     '2009-2010':('Los Angeles Lakers','Boston Celtics'),
                     '2010-11':('Dallas Mavericks','Miami Heat'),
                     '2011-12':('Miami Heat','Oklahoma City Thunder'),
                     '2012-13':('Miami Heat','San Antonio Spurs'),
                     '2013-14':('San Antonio Spurs','Miami Heat'),
                     '2014-15':('Golden State Warriors','Cleveland Cavaliers'),
                     '2015-16':('Cleveland Cavaliers','Golden State Warriors'),
                     '2016-17':('Golden State Warriors','Miami Heat'),
                     '2017-18':('Dallas Mavericks','Cleveland Cavaliers'),
                     '2018-19':('Toronto Raptors','Golden State Warriors'),
                     '2019-20':('Los Angeles Lakers','Miami Heat'),
                     '2020-21':('Milwaukee Bucks','Phoenix Suns'),
                     '2021-22':('Golden State Warriors','Boston Celtics'),
                     '2022-23':('Denver Nuggets','Miami Heat'),}

Para identificarmos os finalistas iremos adicionar uma nova coluna aos dados chamada 'FINAL'. Essa coluna terá valor 0 caso o time não tenha chego a final e valor 1 caso tenha chego a final.

In [77]:
def label(dfSeason, finalistas):
  dfSeason['FINAL'] = 0
  dfSeason['FINAL'] = np.where((dfSeason['TEAM_NAME'] == finalistas[0]) | (dfSeason['TEAM_NAME'] == finalistas[1]), 1, dfSeason['FINAL'])


Aproveitaremos o loop que aplica a função acima em todas as 13 bases de dados para também concatená-las em um dataframe apenas.

In [78]:
#Aplicar a função label e concatenar:

directory = '/content/TPICD/data/basesSeparadas'
dfSeasonTest = pd.read_csv("/content/TPICD/data/basesSeparadas/2010-11.csv")
dfFinal = pd.DataFrame(columns=dfSeasonTest.columns)


for fileName in os.listdir(directory):
    if(fileName != '.ipynb_checkpoints'):
      filePath = os.path.join(directory, fileName)
      dfSeason = pd.read_csv(filePath)
      season = fileName.split('.csv')[0]
      label(dfSeason, seasonFirstSecond[season])
      dfFinal = pd.concat([dfFinal, dfSeason])

dfFinal.to_csv('TemporadasUnificadas.csv')

dfFinal

,Unnamed: 0,TEAM_ID,TEAM_NAME,GP,W,L,W_PCT,MIN,FGM,FGA,...,AST_RANK,TOV_RANK,STL_RANK,BLK_RANK,BLKA_RANK,PF_RANK,PFD_RANK,PTS_RANK,PLUS_MINUS_RANK,FINAL
0,0,1610612737,Atlanta Hawks,72,41,31,0.569,48.3,40.8,87.2,...,19,9,24,17,17,18,9,13,9,0.0
1,1,1610612738,Boston Celtics,72,36,36,0.500,48.3,41.5,88.9,...,25,16,13,6,11,24,13,16,13,0.0
2,2,1610612751,Brooklyn Nets,72,48,24,0.667,48.3,43.1,87.3,...,7,13,27,7,9,13,18,2,7,0.0
3,3,1610612766,Charlotte Hornets,72,33,39,0.458,48.2,39.9,87.8,...,5,24,10,16,14,5,21,23,23,0.0
4,4,1610612741,Chicago Bulls,72,31,41,0.431,48.3,42.2,88.6,...,8,27,28,27,18,10,30,21,20,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25,25,1610612758,Sacramento Kings,82,17,65,0.207,48.5,36.5,81.5,...,27,26,23,25,20,29,11,12,29,0.0
26,26,1610612759,San Antonio Spurs,82,54,28,0.659,48.5,37.1,79.7,...,12,1,30,27,8,1,30,23,7,0.0
27,27,1610612761,Toronto Raptors,82,33,49,0.402,48.3,37.2,81.4,...,5,10,27,16,12,3,22,16,23,0.0
28,28,1610612762,Utah Jazz,82,48,34,0.585,48.5,38.3,80.8,...,1,22,1,21,21,23,1,7,9,0.0


Verificar valores nulos:

In [79]:
dfFinal.isnull().sum()

Unnamed: 0         0
TEAM_ID            0
TEAM_NAME          0
GP                 0
W                  0
L                  0
W_PCT              0
MIN                0
FGM                0
FGA                0
FG_PCT             0
FG3M               0
FG3A               0
FG3_PCT            0
FTM                0
FTA                0
FT_PCT             0
OREB               0
DREB               0
REB                0
AST                0
TOV                0
STL                0
BLK                0
BLKA               0
PF                 0
PFD                0
PTS                0
PLUS_MINUS         0
GP_RANK            0
W_RANK             0
L_RANK             0
W_PCT_RANK         0
MIN_RANK           0
FGM_RANK           0
FGA_RANK           0
FG_PCT_RANK        0
FG3M_RANK          0
FG3A_RANK          0
FG3_PCT_RANK       0
FTM_RANK           0
FTA_RANK           0
FT_PCT_RANK        0
OREB_RANK          0
DREB_RANK          0
REB_RANK           0
AST_RANK           0
TOV_RANK     

###Pré-processamento

Iremos excluir algumas colunas que não irão nos ajudar, são elas: 'Unnamed: 0', 'TEAM_ID', 'TEAM_NAME' e 'GP'     (GP = Games played, igual para todos)

In [80]:
dfFinal = dfFinal.iloc[:,4:]
dfFinal

,W,L,W_PCT,MIN,FGM,FGA,FG_PCT,FG3M,FG3A,FG3_PCT,...,AST_RANK,TOV_RANK,STL_RANK,BLK_RANK,BLKA_RANK,PF_RANK,PFD_RANK,PTS_RANK,PLUS_MINUS_RANK,FINAL
0,41,31,0.569,48.3,40.8,87.2,0.468,12.4,33.4,0.373,...,19,9,24,17,17,18,9,13,9,0.0
1,36,36,0.500,48.3,41.5,88.9,0.466,13.6,36.4,0.374,...,25,16,13,6,11,24,13,16,13,0.0
2,48,24,0.667,48.3,43.1,87.3,0.494,14.2,36.1,0.392,...,7,13,27,7,9,13,18,2,7,0.0
3,33,39,0.458,48.2,39.9,87.8,0.455,13.7,37.0,0.369,...,5,24,10,16,14,5,21,23,23,0.0
4,31,41,0.431,48.3,42.2,88.6,0.476,12.6,34.0,0.370,...,8,27,28,27,18,10,30,21,20,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25,17,65,0.207,48.5,36.5,81.5,0.447,7.1,19.4,0.368,...,27,26,23,25,20,29,11,12,29,0.0
26,54,28,0.659,48.5,37.1,79.7,0.466,7.6,19.8,0.386,...,12,1,30,27,8,1,30,23,7,0.0
27,33,49,0.402,48.3,37.2,81.4,0.458,5.8,15.7,0.372,...,5,10,27,16,12,3,22,16,23,0.0
28,48,34,0.585,48.5,38.3,80.8,0.475,4.8,13.7,0.349,...,1,22,1,21,21,23,1,7,9,0.0


Também iremos separar o dataframe em dois, um que contém as estatísticas e o label de finalista e outro que contém os rankings das características e o label de finalista:

In [81]:
dfEstat = dfFinal.iloc[:,:25]
dfEstat['FINAL'] = dfFinal['FINAL']
dfRank = dfFinal.iloc[:,26:]

Além disso, entre W (wins), L (losses) e W_PCT (win percentage) basta mantermos uma delas, no caso W_PCT será mantida:

In [82]:
dfEstat = dfEstat.iloc[:,2:]
dfRank = dfRank.iloc[:,2:]



In [83]:
dfEstat.head()

,W_PCT,MIN,FGM,FGA,FG_PCT,FG3M,FG3A,FG3_PCT,FTM,FTA,...,AST,TOV,STL,BLK,BLKA,PF,PFD,PTS,PLUS_MINUS,FINAL
0,0.569,48.3,40.8,87.2,0.468,12.4,33.4,0.373,19.7,24.2,...,24.1,13.2,7.0,4.8,5.0,19.3,19.8,113.7,2.3,0.0
1,0.500,48.3,41.5,88.9,0.466,13.6,36.4,0.374,16.1,20.8,...,23.5,14.1,7.7,5.3,4.6,20.4,19.3,112.6,1.5,0.0
2,0.667,48.3,43.1,87.3,0.494,14.2,36.1,0.392,18.1,22.5,...,26.8,13.5,6.7,5.3,4.6,19.0,18.9,118.6,4.5,0.0
3,0.458,48.2,39.9,87.8,0.455,13.7,37.0,0.369,15.9,20.9,...,26.8,14.8,7.8,4.8,4.8,18.0,18.6,109.5,-1.9,0.0
4,0.431,48.3,42.2,88.6,0.476,12.6,34.0,0.370,13.8,17.5,...,26.8,15.1,6.7,4.2,5.1,18.9,17.7,110.7,-0.9,0.0


In [84]:
dfRank.head()

,W_PCT_RANK,MIN_RANK,FGM_RANK,FGA_RANK,FG_PCT_RANK,FG3M_RANK,FG3A_RANK,FG3_PCT_RANK,FTM_RANK,FTA_RANK,...,AST_RANK,TOV_RANK,STL_RANK,BLK_RANK,BLKA_RANK,PF_RANK,PFD_RANK,PTS_RANK,PLUS_MINUS_RANK,FINAL
0,11,11,19,23,17,17,19,12,2,4,...,19,9,24,17,17,18,9,13,9,0.0
1,16,16,13,11,19,11,10,10,25,25,...,25,16,13,6,11,24,13,16,13,0.0
2,4,11,6,21,1,7,12,2,5,8,...,7,13,27,7,9,13,18,2,7,0.0
3,19,19,22,20,22,10,9,14,26,24,...,5,24,10,16,14,5,21,23,23,0.0
4,21,16,10,14,9,16,17,13,30,30,...,8,27,28,27,18,10,30,21,20,0.0


##Análise

Inicialmente iremos buscar correlações entre os dados:

In [85]:
cormat = dfEstat.corr()
round(cormat,2)

,W_PCT,MIN,FGM,FGA,FG_PCT,FG3M,FG3A,FG3_PCT,FTM,FTA,...,AST,TOV,STL,BLK,BLKA,PF,PFD,PTS,PLUS_MINUS,FINAL
W_PCT,1.00,-0.03,0.28,-0.08,0.60,0.20,0.12,0.54,0.15,0.09,...,0.28,-0.31,0.16,0.26,-0.47,-0.20,0.11,0.31,0.97,0.32
MIN,-0.03,1.00,-0.10,-0.08,-0.07,-0.15,-0.15,-0.06,0.04,0.04,...,-0.08,0.05,-0.05,-0.02,0.03,0.10,0.07,-0.12,-0.05,-0.06
FGM,0.28,-0.10,1.00,0.82,0.64,0.71,0.69,0.30,-0.09,-0.19,...,0.78,-0.16,0.20,0.10,-0.13,-0.15,-0.16,0.93,0.29,0.11
FGA,-0.08,-0.08,0.82,1.00,0.09,0.71,0.75,-0.02,-0.23,-0.30,...,0.59,-0.14,0.18,0.02,0.14,-0.13,-0.25,0.78,-0.06,-0.06
FG_PCT,0.60,-0.07,0.64,0.09,1.00,0.28,0.20,0.55,0.16,0.08,...,0.57,-0.08,0.11,0.16,-0.41,-0.09,0.07,0.58,0.61,0.28
FG3M,0.20,-0.15,0.71,0.71,0.28,1.00,0.99,0.33,-0.17,-0.26,...,0.61,-0.15,0.06,-0.04,-0.19,-0.20,-0.19,0.84,0.21,0.10
FG3A,0.12,-0.15,0.69,0.75,0.20,0.99,1.00,0.19,-0.18,-0.25,...,0.59,-0.12,0.08,-0.05,-0.13,-0.19,-0.18,0.82,0.13,0.07
FG3_PCT,0.54,-0.06,0.30,-0.02,0.55,0.33,0.19,1.00,-0.02,-0.12,...,0.30,-0.20,-0.07,0.04,-0.41,-0.08,-0.08,0.33,0.55,0.21
FTM,0.15,0.04,-0.09,-0.23,0.16,-0.17,-0.18,-0.02,1.00,0.93,...,-0.17,0.17,0.00,0.10,0.11,0.38,0.80,0.13,0.17,-0.01
FTA,0.09,0.04,-0.19,-0.30,0.08,-0.26,-0.25,-0.12,0.93,1.00,...,-0.25,0.29,0.03,0.09,0.16,0.41,0.87,0.01,0.10,-0.02


Percebe-se algumas correlações mais altas:

*   FG3M e FG3A = 0.99
*   W_PCT e PLUS_MINUS = 0.96
*   POINTS e FGM = 0.93
*   FTM e FTA = 0.91

In [86]:
#Excluir correlação entre as estatísticas:

dfEstatComp = dfEstat
dfRankComp = dfRank


dfEstat = dfEstat.drop(['FG3A', 'PLUS_MINUS', 'FGM', 'FTA'], axis=1)
dfRank = dfRank.drop(['FG3A_RANK', 'PLUS_MINUS_RANK', 'FGM_RANK', 'FTA_RANK'], axis=1)

###Regressão

O nosso objetivo é prever se um time irá chegar à final dos playoffs utilizando seus dados da temporada regular. Como o esporte evolui bastante com os anos, iremos utilizar o dataset que contém o ranking das estatísticas, pois dessa forma mantemos a ordem das estatísticas entre os times mas reduzimos o fator de mudança das médias das estatísticas com os anos.

Como a variável que queremos prever é categórica (FINAL = 0 ou 1) iremos utilizar a regressão logística:

In [87]:
y = dfRank['FINAL']
xRank = dfRank.iloc[:,:-1]

xTrain, xTest, yTrain, yTest = train_test_split(xRank, y, test_size=0.2, shuffle=True)

Inicialmente iremos aplicar a regressão logística de forma simples:

In [88]:
logRegRank = LogisticRegression(max_iter=1000)
logRegRank.fit(xTrain,yTrain)

predicts1 = logRegRank.predict(xTest)
confusion_matrix(yTest,predicts1)

array([[91,  0],
       [ 9,  2]])

Análise com os ranks mais importantes de acordo com o outro colab:

In [89]:
df = dfRank
x = xTrain[['W_PCT_RANK','BLKA_RANK','FG_PCT_RANK','DREB_RANK','PTS_RANK','AST_RANK','FG3_PCT_RANK','BLK_RANK']]
y = yTrain

logReg3 = LogisticRegression(max_iter=1000)
logReg3.fit(x,y)

predicts2 = logReg3.predict(xTest[['W_PCT_RANK','BLKA_RANK','FG_PCT_RANK','DREB_RANK','PTS_RANK','AST_RANK','FG3_PCT_RANK','BLK_RANK']])
confusion_matrix(yTest,predicts2)

array([[91,  0],
       [11,  0]])

Agora iremos aplicar validação cruzada sobre modelos que utilizam regressão logística:

In [90]:
model = GridSearchCV(LogisticRegression(),
                     param_grid={'penalty': ['l1', 'l2'], 'C': [0.001, 0.01, 0.1, 1, 10, 100], 'max_iter': [1000]},
                     cv = 5,
                     refit=True)

model = model.fit(xTrain, yTrain)


/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_validation.py:378: FitFailedWarning: 
30 fits failed out of a total of 60.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
30 fits failed with the following error:
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_validation.py", line 686, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py", line 1162, in fit
    solver = _check_solver(self.solver, self.penalty, self.dual)
  File "/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py", line 54, in _check_solver
    raise ValueError(
ValueError: Solve

In [91]:
predicts3 = model.predict(xTest)
confusion_matrix(yTest,predicts3)


array([[91,  0],
       [11,  0]])

Também iremos experimentar treinar a o modelo informando que o dataset está desbalanceado:

In [92]:
model = GridSearchCV(LogisticRegression(),
                     param_grid={'penalty': ['l1', 'l2'], 'C': [0.001, 0.01, 0.1, 1, 10, 100], 'max_iter': [1000], 'class_weight': ['balanced']},
                     cv = 5,
                     refit=True)

model = model.fit(xTrain, yTrain)

/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_validation.py:378: FitFailedWarning: 
30 fits failed out of a total of 60.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
30 fits failed with the following error:
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_validation.py", line 686, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py", line 1162, in fit
    solver = _check_solver(self.solver, self.penalty, self.dual)
  File "/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py", line 54, in _check_solver
    raise ValueError(
ValueError: Solve

In [93]:
predicts4 = model.predict(xTest)
confusion_matrix(yTest,predicts4)

array([[71, 20],
       [ 3,  8]])

Conclusão:

Observando as 4 matrizes de confusão obtidas acima percebemos que os resultados não são muito animadores. O principal motivo parece ser o desbalanceamento do dataset, no entanto, ao informar o modelo disso, aumentamos bastante o número de falsos positivos, que, nesse caso, é bem ruim.

Além do desbalanceamento do dataset, tentar prever apenas se um time chegará ou não à final não pareceu ter sido uma boa escolha. Pois deixamos de avaliar o quão perto da final os times chegaram. Por causa disso, optamos por fazer a próxima análise tentando prever o número de vitórias nos playoffs, ao invés de prever se um time chegará ou não à final.